<a href="https://colab.research.google.com/github/Mraghuvaran/Regreession-models/blob/master/income_classification_pipelining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np 
import pandas as pd 

In [0]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Income Dataset/train_data.csv")
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Income Dataset/test_data.csv")

In [5]:
train.head()

,index,age,working_sector,financial_weight,qualification,years_of_education,tax_paid,loan_taken,marital_status,occupation,relationship,ethnicity,gender,gain,loss,working_hours,country,target
0,1,39.0,state,77516.0,Bachelors,13,NaN,0,Never-married,clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,0
1,2,50.0,self_employed,83311.0,Bachelors,13,NaN,0,Married-civilian,managerial,Husband,White,Male,0.0,0.0,13.0,United-States,0
2,3,38.0,private,215646.0,HS-grad,9,NaN,0,Divorced,cleaner,Not-in-family,White,Male,0.0,0.0,40.0,United-States,0
3,4,53.0,private,234721.0,11th,7,NaN,0,Married-civilian,cleaner,Husband,Black,Male,0.0,0.0,40.0,United-States,0
4,5,28.0,private,338409.0,Bachelors,13,NaN,0,Married-civilian,specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,0


In [6]:
test.head()

,index,age,working_sector,financial_weight,qualification,years_of_education,tax_paid,loan_taken,marital_status,occupation,relationship,ethnicity,gender,gain,loss,working_hours,country
0,31588,28,private,183175,Some-college,10,5491.305588,1,Divorced,clerical,Not-in-family,White,Female,0,0,40,United-States
1,31589,29,local_body,115585,Some-college,10,3456.560000,0,Never-married,cleaner,Not-in-family,White,Male,0,0,50,United-States
2,31590,37,private,254202,Bachelors,13,3321.670000,0,Married-civilian,sales,Husband,White,Male,0,0,50,United-States
3,31591,71,self_employed,494223,Some-college,10,NaN,0,Separated,sales,Unmarried,Black,Male,0,1816,2,United-States
4,31592,27,private,253814,HS-grad,9,4840.481164,0,Married-non-resident,sales,Unmarried,White,Female,0,0,25,United-States


In [0]:
#train.gain.value_counts()

In [0]:
#dropping the columns which are not useful 
train.drop(columns= ['index','tax_paid'], axis=1, inplace=True)


#test data columns removal 

test.drop(columns= ['index','tax_paid'], axis=1, inplace=True)



In [9]:
train.shape

(31587, 16)

In [10]:
test.shape

(976, 15)

In [11]:
train.dtypes

age                   float64
working_sector         object
financial_weight      float64
qualification          object
years_of_education      int64
loan_taken              int64
marital_status         object
occupation             object
relationship           object
ethnicity              object
gender                 object
gain                  float64
loss                  float64
working_hours         float64
country                object
target                  int64
dtype: object

In [0]:
#converting objects to categories 
for col in ['working_sector','qualification','years_of_education','loan_taken','marital_status','occupation','relationship','ethnicity','gender',
            'country','target']:
  train[col] = train[col].astype('category')


In [13]:
cat_attr = list(train.select_dtypes("category").columns)
num_attr = list(train.columns.difference(cat_attr))

cat_attr.pop()

'target'

In [14]:
cat_attr

['working_sector',
 'qualification',
 'years_of_education',
 'loan_taken',
 'marital_status',
 'occupation',
 'relationship',
 'ethnicity',
 'gender',
 'country']

In [15]:
num_attr

['age', 'financial_weight', 'gain', 'loss', 'working_hours']

In [16]:
# print columns with missing values
missing_cols = train.columns[train.isnull().any()]
print(missing_cols)

Index(['age', 'working_sector', 'financial_weight', 'qualification',
       'occupation', 'relationship', 'ethnicity', 'gender', 'gain', 'loss',
       'working_hours', 'country'],
      dtype='object')


In [17]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.impute import SimpleImputer

from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, recall_score, precision_score

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])


categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_attr),
        ('cat', categorical_transformer, cat_attr)])

In [0]:
clf_logreg = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

In [0]:
#splitting the dat into train & validation data 
X_train, Y_train = train.loc[:,train.columns!='target'], train.loc[:,'target']

X_test = test

In [21]:
X_train.head()

,age,working_sector,financial_weight,qualification,years_of_education,loan_taken,marital_status,occupation,relationship,ethnicity,gender,gain,loss,working_hours,country
0,39.0,state,77516.0,Bachelors,13,0,Never-married,clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States
1,50.0,self_employed,83311.0,Bachelors,13,0,Married-civilian,managerial,Husband,White,Male,0.0,0.0,13.0,United-States
2,38.0,private,215646.0,HS-grad,9,0,Divorced,cleaner,Not-in-family,White,Male,0.0,0.0,40.0,United-States
3,53.0,private,234721.0,11th,7,0,Married-civilian,cleaner,Husband,Black,Male,0.0,0.0,40.0,United-States
4,28.0,private,338409.0,Bachelors,13,0,Married-civilian,specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba


In [0]:
#splitting the train data into validation part. 
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, random_state=12, stratify =Y_train)

In [23]:
### _Build Logistic Regression Model - 1_

clf_logreg.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                             

In [24]:
train_pred = clf_logreg.predict(x_train)
test_pred = clf_logreg.predict(x_test)

print(accuracy_score(y_true= y_train, y_pred=train_pred))
print(accuracy_score(y_true=y_test, y_pred = test_pred))

0.8521317011397214
0.8484234519437761


# Building the SVM classifier-1

In [0]:
from sklearn.svm import SVC

In [26]:

%%time
clf_svm = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', svm.SVC())])

svm_param_grid = {'classifier__C':[0.2,.3],
                  'classifie__kernel':['linear']}
svm_grid = GridSearchCV(clf_svm, param_grid= svm_param_grid,n_jobs=-1, cv=5)

svm_grid.fit(x_train, y_train)



ValueError: ignored

In [0]:

train_svm_pred = svm_grid(x_train)
test_svm_pred = svm_grid(x_test)

print(svm_grid.score(x_train, y_train))
print(svm_grid.score(x_test, y_test))

# _Build Decision Tree Model - 2_


In [0]:

%%time
clf_dt = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', DecisionTreeClassifier())])

dt_param_grid = {'classifier__criterion': ['entropy', 'gini'], 'classifier__max_depth': [5,6,7,8,9,10], 
                 "classifier__min_samples_split": [2,5],"classifier__min_samples_leaf": [1,3,5]}

dt_grid = GridSearchCV(clf_dt, param_grid=dt_param_grid, cv=5)

dt_grid.fit(x_train,y_train)

print(dt_grid.best_params_)

train_pred = dt_grid.predict(x_train)
test_pred = dt_grid.predict(x_test)

print(dt_grid.score(x_train, y_train))
print(dt_grid.score(x_test, y_test))


## _Build Random Forest Model - 3_ (Using Stratified KFold)

###__Stratified K-Folds cross-validator__

## This cross-validation object is a **variation** of KFold that returns stratified folds. The folds are made by **preserving the percentage of samples for each class**.


In [0]:

%%time
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())])

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=143)

param_grid = {"classifier__n_estimators" : [30,40,50],
              "classifier__max_depth" : [2,3,5,6],
              "classifier__max_features" : [3, 5, 7],
              "classifier__min_samples_leaf" : [4, 6, 8, 10]}

rf_grid = GridSearchCV(clf, param_grid=dt_param_grid, cv=kfold)


rf_grid.fit(x_train,y_train)

rf_grid.best_params_

train_pred = rf_grid.predict(x_train)
test_pred = rf_grid.predict(x_test)

print(rf_grid.score(x_train, y_train))
print(rf_grid.score(x_test, y_test))



In [27]:
# !pip install xgboost


In [0]:
import xgboost as xgb

In [0]:
from sklearn.model_selection import GridSearchCV

In [38]:
#XGboosting 
%%time
clf = Pipeline(steps=[('preprocessor', preprocessor),
               ('classifier', xgb.XGBClassifier())])


kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=143)

param_grid = {"classifier__n_estimators" : [70,100],
              "classifier__max_depth" : [3,5,6,7],
              "classifier__colsample_bytree":[0.7,.8],
              "classifier__learning_rate": [0.001,0.01,0.1],
              "classifier__subsample":[0.8,0.6]}

xg_grid = GridSearchCV(clf, param_grid=param_grid, n_jobs=-1, cv=kfold)


xg_grid.fit(x_train,y_train)



train_pred = xg_grid.predict(x_train)
test_pred = xg_grid.predict(x_test)

print(xg_grid.score(x_train, y_train))
print(xg_grid.score(x_test, y_test))


0.8925707049387928
0.8703305052551602
CPU times: user 20.6 s, sys: 1.44 s, total: 22 s
Wall time: 25min 44s


In [39]:
print(xg_grid.best_params_)

{'classifier__colsample_bytree': 0.7, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 7, 'classifier__n_estimators': 120, 'classifier__subsample': 0.6}


In [42]:
#testing and writin gthe values on the test data

xg_grid_test = xg_grid.predict(X_test)

print(xg_grid_test.shape)

xg_results = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Income Dataset/SampleSubmission.csv")

xg_results['target'] = xg_grid_test

xg_results.to_csv("xg_prediction2.csv", index=False)

(976,)


#  _Build Gradient Boosting - 4_

In [0]:

%%time
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('GBM',GradientBoostingClassifier())])

gbm_param_grid = {'GBM__max_depth': [2,3,4], 'GBM__subsample': [0.8, 0.6,], 'GBM__max_features':[0.2, 0.3], 
              'GBM__n_estimators': [10, 20, 30]}

gbm_grid = GridSearchCV(clf, param_grid=gbm_param_grid, cv=3)

gbm_grid.fit(x_train,y_train)

print(gbm_grid.best_params_)

train_pred = gbm_grid.predict(x_train)
test_pred = gbm_grid.predict(x_test)

print(gbm_grid.score(x_train, y_train))
print(gbm_grid.score(x_test, y_test))


{'GBM__max_depth': 4, 'GBM__max_features': 0.2, 'GBM__n_estimators': 30, 'GBM__subsample': 0.8}
0.8593077247783875
0.8555147524376345
CPU times: user 1min 52s, sys: 459 ms, total: 1min 53s
Wall time: 1min 53s


In [0]:
final_pred = gbm_grid.predict(X_test)

In [0]:
final_dt_reslut = dt_grid.predict(X_test)

In [0]:
final_dt_reslut

In [0]:
final_pred

In [0]:

final_pred = pd.Series(final_pred)


In [0]:
dt_reslut = pd.Series(final_dt_reslut)

dt_reslut.rename(columns= {"0":"target"})

final_dt_results.head()

final_dt_results = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Income Dataset/SampleSubmission.csv")

In [0]:
final_dt_results['target']=dt_reslut
#final_submission = pd.concat(final_results, final_pred, join="inner")

In [0]:
final_dt_results.head()

In [0]:
final_dt_results.to_csv("FinalSubmission_dt.csv", index= False)